# Modulus of Permutations Code

In [ ]:
# install packages
%pip install matplotlib
%pip install networkx
%pip install numpy
!brew instal gmp
# cdd installs successfully on windows machine
# %pip install pycddlib
%pip install cvxpy
%pip install nbconvert
# %pip install RISE

In [ ]:
# import packages
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
# import cdd
import math
import cvxpy as cvx
# from collections import deque
import random

___

### General Timeline/Contents

Meeting Notes

Theory

Find all MISS

Find all MDSS

Find MISS with minimum weights (Atallah et al.)

Basic algorithm

Formulate associated graph of paths and cuts

___

### Meeting Notes

##### Jan 29
A set is *convex* if the segment connecting any two points that exist in the set is also entirely contained in the set. i.e. $S \text{ is convex if } \forall x, y \in S, \: \forall t \in [0, 1], \: tx + (1-t)y \in S$.
(affine if this is true for lines, not segments)

A set is *recessive* if any positive addition to the set is still contained in the set.

A *convex hull* of a set $S$ is the smallest convex set containing all points of the $S$, or the intersection of all convex sets containing $S$.

A point $x \in S$ is an *extreme point* of $S$ if for any linear combination of points that define $x$, those points are identically $x$. i.e. $x = \frac{1}{2} x_1 + \frac{1}{2} x_2 \text{ for } x_1, x_2 \in S \text{ and } t \in (0, 1) \implies x = x_1 = x_2$
For a square, these points are the vertices. For a half-disc, these points are the entire arc. Taking the convex combination (linear combination where all coefficients are nonnegative and sum to one) of these extreme points yields the convex hull of the set.

A *face* of a set $S$ denoted $S'$ is defined as a convex subset of $S$ where, if a relative interior point of a linear segment is in $S'$, the endpoints of that linear segment are also in $S'$. These can be n-dimensional. An *extreme direction* is a half-line face of the set.

A *convex cone* $C$ is defined as, if $x, y \in C \text{ and } a, b \text{ are positive scalars}, \: ax + by \in C $. Also, $C + C \subseteq C$

$C = \text{co}(\text{Ext}(C))$ if C is closed, convex, and bounded. If unbounded and in general **Theorem 14.2** applies. $C = \text{co}(\text{Ext}(C)) + \text{cone}(\text{Dir}(C))$

##### Feb 5

More on **Theorem 14.2**: $\text{Co(Ext(A))} \subset A $ \
Assume that $\exists a \in A \setminus \text{Co(Ext(A))}$. Then there exists a hyperplane that divides set $A$ from $a$. Define this hyperplane as $f(x) = c$. Consider $\max_{x \in A} f(x)$ then $\exists x_0 \in \text{ Ext(A) s.t. } x_0 \text{ is optimal and maximizes } f(x).$ OTOH, $f(b) < c < f(a) \: \forall b \in \text{Co(Ext(A))}$ $\Rightarrow \! \Leftarrow$ 

$\text{Dom(A)}$ is the closure of the convex hull with any positive vector (up and right)

$\text{Adm}(\Gamma)$ is the family of admissible objects an object is admissible if $\mathcal{N}(\gamma, \cdot)^T \rho \geq 1$. The admissible set is intersection of all the constraints ("intersection of closed half spaces"), along with $\rho \geq 0$

$\hat{\Gamma} := \text{Ext(Adm}(\Gamma))$ is the *Fulkerson Blocker (dual family)* of $\Gamma$

If $\mathcal{N} = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix}$, the admissible set is the intersection of $y \geq 1$ and $x \geq 1$. The extreme point of this set is $(1,1)$. Thus, $\hat{\mathcal{N}} = \begin{bmatrix} 1 & 1 \end{bmatrix}$. Proving the "dual," $\text{Adm}(\hat{\Gamma})$ is the intersection of the positive quadrant with $x + y \geq 1$. The extreme points of this set are $(1,0)$ and $(0,1)$

The supp($\gamma$) are the columns that are '1' in the row vector corresponding to used edges. A family is proper if $\nexists \: \gamma_1, \gamma_2 \in \Gamma$ s.t. supp($\gamma_1$) $\subsetneq$ supp($\gamma_2$) 

##### Feb 12

To generate the associated graph of directed paths, we draw a directed edge from each node to each other node as long as $\text{val}[a] < \text{val}[b]$. We then delete edges with intermediate nodes. MISS would be all the paths from source to sink. Then we could find extreme points and directions as in the modulus book (using cdd)

Remember the output of find_MISS needs to be a usage matrix

Things to include: \
convex sets \
chapters in books \
fulkerson duality \
permutation \
young tableaux \
read paper, find MISS with minimum weights (paper) \
basic algorithm \
formulate associated graph (paths and cuts) like above (cdd)

##### Feb 19

Implement algorithm from last week, finding paths from sources to sinks. Find sources and sinks by looping through all neighbors of points and seeing if any are left and less or right and greater
\ also finish recursive function

From paper (Atallah et al.): \
Algorithm for finding minumium independent dominating set in a permutation graph (equivalent to finding the shortest maximal increasing subsequence) \
*permutation graph*, *dominating set*, and *independent set* are defined in theory section below

Also discussed
$\epsilon \text{-nets}$ - given radius $\epsilon$, balls of radius $\epsilon$ need to cover the set. The net must be minimal i.e. every ball must be essential.

Also that an *independent dominating set* is *minimal dominating* since adding a node would make it not independent 

Also that maximal increasing subsequences in $\pi$ are an independent dominating set of $G(\pi)$

Suppose $A \in$ MIS, $A$ is subset of $V$, $A$ is an IDS

Assume A is not dominating. Then $\exists v \text{ s.t. } v \text{ in some } i_j < i_{j+1} \text{ s.t. } \pi(v) > \pi(i_j) \text{ and } \pi(v) < \pi(i_{j+1})$

Note: \
Maximal corresponds to dominating and increasing corresponds to independent, maximal independent is dominating

if $A$ is not independent, then $\exists a,b \in A \text{ where } a \sim b \text{ (adjacent) }, a < b \text{ s.t. } \pi(a) > \pi(b) \rightarrow A \text{ is not increasing }$

Since this is equivalent to maximal independent sets, we could also create a graph and use networkx to find maximal independent sets (These can be generated by picking a node, adding it to the set, then finding one with no neighbors with the current set of nodes, the repeating until no nodes can be added to the set).

Also add some comments

##### Feb 26

Todo list:
Do a timeit to time both algos \
Figure out `cdd` installation and implementation \
Compute matrix difference to see if both algorithms give same result \
Look at dynamic programming for recursive algorithm \
Run through an example for the function in a markdown file \
Try to make algo faster \
Give each node a weight in recursive algorithm &rarr; find lightest dot product of a row in N with the weight vector &rarr; at return, record a sum compare with previous and then find least weight \
Implement basic algorithm after getting lightest row \
Then implement +1 algorithm and compare times \
Could then make custom basic algorithm, do basic algorithm while computing through recursive algorithm, etc

basic algorithm notes:

takes first row in N, puts in N'. Then computes weights vector rho such that that row * weights = 1. Then appends next row to N', i.e. N' is now the first two rows of N and uses convex optimization to find weights vector rho such that both vectors in N * rho yields 1. Continue until find rho such that all rows of N yield rho

##### Mar 4

Keep implementing basic algorithm, +1 algorithm \
Details of implementation in paper, details of details in cited paper (augmented tree structure, etc) \
Use cdd on desktop and save result for laptop viewing \
Make poster?

##### Mar 18

Keep working on basic algorithm

##### Mar 25



___

### Theory 

A permutation is an ordering of elements

Given a permutation, the *permutation graph* of that permutation is the undirected graph with vertices for each element such that a vertex $i$ is adjacent to another vertex $j \text{ iff } i < j$ and $j$ comes before $i$ in the permutation

A *dominating set* is a subset of vertices in a graph such that any vertex in the original graph is in the subset or adjacent to a vertex in the subset

A set is *independent* if no two verices in the set are adjacent

Given a sequence $a = a_1 a_2 ... a_n$, a subsequence is a sequence of elements in a such that indicies $i_1 < i_2 < \dots < i_n$. This sequence is *increasing* if additionally, $a_{i_1} < a_{i_2} < \dots < a_{i_n}$.

A sequence is *maximal* if there it is not a part of a another, longer maximal increasing subsequence (MISS). A sequence is *maximum* if it is of maximum length of all the *maximal* increasing subsequences.

From Atallah et al. *$DOM(p)$* contains the points below and left of $p$. *$MAX(P)$* denote the maxima of set $P$.

___

### Theory for DRP Talk

A permutation is an ordering of elements

Given a sequence $a = a_1 a_2 ... a_n$, a subsequence is a sequence of elements in a such that indicies $i_1 < i_2 < \dots < i_n$. This sequence is *increasing* if additionally, $a_{i_1} < a_{i_2} < \dots < a_{i_n}$.

A sequence is *maximal* if there it is not a part of a another, longer maximal increasing subsequence (MISS). A sequence is *maximum* if it is of maximum length of all the *maximal* increasing subsequences.

Consider $\Gamma$ to be a collection (family) of objects. In this case, these objects are maximal increasing subsequences. Each family $\Gamma$ has objects $\gamma$. We can keep track of these objects in a *usage matrix* $\mathcal{N}$ with entries defined as $\mathcal{N}(\gamma, j) := \begin{cases} 1 & \text{if} \: j = \: \text{a point in the MISS} \\ 0 & \text{otherwise} \end{cases}$

Given a density $\rho \in \mathbb{R}_{+}^{N}$, define the $\rho$-length to be $\ell_\rho(\gamma) := \sum_{j=1}^{N} \mathcal{N}(\gamma, j)\rho(j)$

A density $\rho$ is admissible for $\Gamma$ if $\ell_\rho(\gamma) \geq 1 \: \forall \: \gamma \in \Gamma$ and such $\rho \in \text{Adm}(\Gamma)$

Define the usual (2)-modulus as $\text{Mod}(\Gamma) = \inf\limits_{\rho \in \text{Adm}(\Gamma)} \sum_{j=1}^{N} \rho(j)^2$

Similarly, the 1-modulus is $\text{Mod}(\Gamma) = \inf\limits_{\rho \in \text{Adm}(\Gamma)} \sum_{j=1}^{N} \rho(j)$

There are two algorithms for computing this modulus that I have tried: The Basic Algorithm and the Plus-1 Algorithm

For these algorithms to work, we neeed to find the $\rho$-lightest MISS in the set. The paper by Atallah et al. gives a good algorithm for finding these


___
### Find all MISS - Examples

Consider the permutation of $1,\dots,5$:

$$
\pi = 2\ 1\ 4\ 5\ 3
$$

In [ ]:
pi =  [2, 1, 4, 5, 3]
x = range(1,6)

# initialize figure
fig, fig = plt.subplots(figsize=(4, 4))
    
plt.scatter(x,pi)

# Set the ticks to be at the edges of the bins.
fig.set_xticks(range(0,6))
fig.set_yticks(range(0,6))
plt.grid(True)

Here's an example of a **MISS** (this one is also maximum):
$$
1\ 4\ 5
$$
corresponding to the indicies $2<3<4$.

In [ ]:
pi =  [2, 1, 4, 5, 3]
x = range(1,6)

ga = [1, 4, 5]
I = [2, 3, 4]

# initialize figure
fig, fig = plt.subplots(figsize=(4, 4))
    
plt.scatter(x,pi)
plt.plot(I,ga)

# Set the ticks to be at the edges of the bins.
fig.set_xticks(range(0,6))
fig.set_yticks(range(0,6))
plt.grid(True)

Now to find all MISS. Note that the resulting graph will be a subgraph of the graph obtained by saying that two points $(i_1,\pi(i_1))$ and $(i_2,\pi(i_2))$ are connected iff $(i_2-i_1)(\pi(i_2)-\pi(i_1))>0$, because the latter will include also increasing subsequences that are not maximal.

In [ ]:
def plot_all_connections(s):
    """returns all valid connections in MISS

    Args:
        s (list): sequence to generate connections from

    Returns:
        list of tuples: (x1, x2) where (x1, s[x1]) (x2, s[x2]) is a valid connection of up and right paths that are part of a MISS
    """
    connections = []
    for i in range(len(s)):
        for j in range(i, len(s)):
            # using the above rule
            if ((j - i) * (s[j] - s[i])) > 0:
                connections.append((i, j))
    # gets rid of duplicates
    connections = list(set(connections))

    plt.figure(figsize=(4, 4))
    plt.scatter(range(1, len(s) + 1), s)

    valid_connections = set()

    for connection in connections:
        valid_connections.add((connection[0] + 1, connection[1] + 1))
        x1, x2 = connection[0], connection[1]
        plt.plot([x1 + 1, x2 + 1], [s[x1], s[x2]], color='tan')

    plt.xticks(range(0, len(s) + 1))
    plt.yticks(range(0, len(s) + 1))
    plt.grid(True)

    connections = sorted(list(valid_connections))

    return connections

print(plot_all_connections(pi))

In [ ]:
# Some failed attempts: 

def plot_all_MISS(s):
    MISSi = []
    seen = []
    for i in range(len(s)):
        if i not in seen:
            temp = i
            ISSi = [i]
            for j in range(i, len(s)):
                if j not in seen:
                    if s[j] > s[temp]:
                        ISSi.append(j)
                        temp = j
                        seen.append(j)
            MISSi.append(ISSi)

    fig, ax = plt.subplots(figsize=(8, 8))
    plt.scatter(range(1, 21), s)

    MISS = [[s[i] for i in sublist] for sublist in MISSi]
    MISSi = [[i + 1 for i in sublist] for sublist in MISSi]

    for x, y in zip(MISSi, MISS):
        plt.plot(x, y, color='blue')
    ax.set_xticks(range(0,21))
    ax.set_yticks(range(0,21))
    plt.grid(True)

    return MISS


def plot_all_MISS(s):
    MISSi = []
    for i in range(len(s)):
        temp = i
        ISSi = [i]
        for j in range(i, len(s)):
            if s[j] > s[temp]:
                ISSi.append(j)
                temp = j
        MISSi.append(ISSi)

    fig, ax = plt.subplots(figsize=(8, 8))
    plt.scatter(range(1, 21), s)

    MISS = [[s[i] for i in sublist] for sublist in MISSi]
    MISSi = [[i + 1 for i in sublist] for sublist in MISSi]

    for x, y in zip(MISSi, MISS):
        plt.plot(x, y, color='blue')
    ax.set_xticks(range(0,21))
    ax.set_yticks(range(0,21))
    plt.grid(True)

    return MISS

def plot_all_MISS(s):
    MISSi = []
    for i in range(1, 2):
        seen = set()
        while any(x > i and s[x] > s[i] and x not in seen for x in range(i+1, len(s))):
            ISSi = [i]
            temp = i
            for j in range(i, len(s)):
                if j not in seen:
                    if s[j] > s[temp]:
                        ISSi.append(j)
                        temp = j
                        seen.add(j)
            MISSi.append(ISSi)

    fig, ax = plt.subplots(figsize=(8, 8))
    plt.scatter(range(1, 21), s)

    MISS = [[s[i] for i in sublist] for sublist in MISSi]
    MISSi = [[i + 1 for i in sublist] for sublist in MISSi]

    for x, y in zip(MISSi, MISS):
        plt.plot(x, y, color='blue')
    ax.set_xticks(range(0,21))
    ax.set_yticks(range(0,21))
    plt.grid(True)

    return MISS

# maybe recursively


In [ ]:
def in_box(t, s):
    """returns whether there is another point in the box extending horizontal and vertical lines from both points (shortcuts)

    Args:
        t (tuple): connection to be checked
        s (list): main seequence

    Returns:
        bool: whether there exists a point in the box
    """
    i, j = t[0], t[1]
    if any(s[x] < s[j] and s[x] > s[i] for x in range(i + 1, j)):
        return True
    return False

In [ ]:
def plot_all_connections(s):
    """returns all valid connections in MISS

    Args:
        s (list): sequence to generate connections from

    Returns:
        list of tuples: (x1, x2) where (x1, s[x1]) (x2, s[x2]) is a valid connection of up and right paths that are part of a MISS
    """
    connections = []
    for i in range(len(s)):
        for j in range(i, len(s)):
            # using the above rule
            if ((j - i) * (s[j] - s[i])) > 0:
                connections.append((i, j))
    # gets rid of duplicates
    connections = list(set(connections))

    plt.figure(figsize=(4, 4))
    plt.scatter(range(1, len(s) + 1), s)

    valid_connections = set()

    for connection in connections:
        if not in_box(connection, s):
            valid_connections.add((connection[0] + 1, connection[1] + 1))
            x1, x2 = connection[0], connection[1]
            plt.plot([x1 + 1, x2 + 1], [s[x1], s[x2]], color='tan')

    plt.xticks(range(0, len(s) + 1))
    plt.yticks(range(0, len(s) + 1))
    plt.grid(True)

    connections = sorted(list(valid_connections))

    return connections


def in_box(t, s):
    """returns whether there is another point in the box extending horizontal and vertical lines from both points

    Args:
        t (tuple): connection to be checked
        s (list): main seequence

    Returns:
        bool: whether there exists a point in the box
    """
    i, j = t[0], t[1]
    if any(s[x] < s[j] and s[x] > s[i] for x in range(i + 1, j)):
        return True
    return False


def MISS_from_connections(c, s_len):
    """uses networkx to find the usage matrix of MISS from a list of possible valid connections

    Args:
        c (list of tuples): connections

    Returns:
        2D numpy array: usage matrix
    """
    plt.figure(figsize=(4, 4))
    g = nx.DiGraph()
    g.add_edges_from(c)
    
    m = np.empty((0, s_len))
        
    sink_nodes = [node for node, out_degree in g.out_degree() if out_degree == 0]
    source_nodes = [node for node, in_degree in g.in_degree() if in_degree == 0]
    for (source, sink) in [(source, sink) for sink in sink_nodes for source in source_nodes]:
        for path in nx.all_simple_paths(g, source=source, target=sink):
            row = np.zeros(s_len)
            for i in path:
                row[i - 1] = 1
            m = np.concatenate((m, [row]), axis=0)

    pos = nx.circular_layout(g)
    nx.draw(g, pos, with_labels=True, arrowsize=20, node_size=700, node_color='skyblue')

    return m

pi = [2, 1, 4, 5, 3]

print("connections:")
connections = plot_all_connections(pi)
print(connections)
print("usage matrix from connections and networkx:")
print(MISS_from_connections(connections, len(pi)))

In [ ]:
def plot_all_connections(s):
    """returns all valid connections in MISS

    Args:
        s (list): sequence to generate connections from

    Returns:
        list of tuples: (x1, x2) where (x1, s[x1]) (x2, s[x2]) is a valid connection of up and right paths that are part of a MISS
    """
    connections = []
    for i in range(len(s)):
        for j in range(i, len(s)):
            # using the above rule
            if ((j - i) * (s[j] - s[i])) > 0:
                connections.append((i, j))
    # gets rid of duplicates
    connections = list(set(connections))

    plt.figure(figsize=(4, 4))
    plt.scatter(range(1, len(s) + 1), s)

    valid_connections = set()

    for connection in connections:
        if not in_box(connection, s):
            valid_connections.add((connection[0] + 1, connection[1] + 1))
            x1, x2 = connection[0], connection[1]
            plt.plot([x1 + 1, x2 + 1], [s[x1], s[x2]], color='tan')

    plt.xticks(range(0, len(s) + 1))
    plt.yticks(range(0, len(s) + 1))
    plt.grid(True)

    connections = sorted(list(valid_connections))

    return connections


def in_box(t, s):
    """returns whether there is another point in the box extending horizontal and vertical lines from both points

    Args:
        t (tuple): connection to be checked
        s (list): main seequence

    Returns:
        bool: whether there exists a point in the box
    """
    i, j = t[0], t[1]
    if any(s[x] < s[j] and s[x] > s[i] for x in range(i + 1, j)):
        return True
    return False

def find_sources(s):
    """finds sources of a sequence (all sources of MISS)

    Args:
        s (list): main sequence

    Returns:
        list: indices of source nodes
    """
    return [i for i, element in enumerate(s) if all(element < x for x in s[:i])]

def plot_all_MISS(s):
    """calls recursive function to find all MISS

    Args:
        s (list): main sequence

    Returns:
        2D numpy array: usage matrix MISS
    """
    m = np.empty((0, len(s)))
    def recurse(s, x, row):
        row[x] = 1
        greater = []
        for i in range(x, len(s)):
            # print(f"i {i}")
            # print(s[i], s[x])
            if s[i] > s[x] and all(s[j] > s[i] for j in greater):
                greater.append(i)
            # print(greater)
        # greater = [i for i in range(x, len(s)) if s[i] > s[x] and not any(s[j] < s[i] for j in range(x + 1, i))]
        # print("greater:")
        # print(greater)
        # print("s[greater]:")
        # print([s[i] for i in greater])
        if len(greater) == 0:
            nonlocal m
            m = np.concatenate((m, [row]), axis=0)
            # print(f"m:\n {m}")
            return
        for element in greater:
            recurse(s, element, row.copy())

    for source in find_sources(s):
        # print(source)
        recurse(s, source, np.zeros(len(s))) 

    plt.figure(figsize=(4, 4))

    plt.scatter(range(1, len(s) + 1), s)
    
    for row in m:
        indices = np.nonzero(row)
        plt.plot(indices[0] + 1, [s[i] for i in indices[0]], color='tan')

    plt.xticks(range(0, len(s) + 1))
    plt.yticks(range(0, len(s) + 1))
    plt.grid(True)

    return m

def MISS_from_connections(c, s_len):
    """uses networkx to find the usage matrix of MISS from a list of possible valid connections

    Args:
        c (list of tuples): connections

    Returns:
        2D numpy array: usage matrix
    """
    plt.figure(figsize=(4, 4))
    g = nx.DiGraph()
    g.add_edges_from(c)
    
    m = np.empty((0, s_len))
        
    sink_nodes = [node for node, out_degree in g.out_degree() if out_degree == 0]
    source_nodes = [node for node, in_degree in g.in_degree() if in_degree == 0]
    for (source, sink) in [(source, sink) for sink in sink_nodes for source in source_nodes]:
        for path in nx.all_simple_paths(g, source=source, target=sink):
            row = np.zeros(s_len)
            for i in path:
                row[i - 1] = 1
            m = np.concatenate((m, [row]), axis=0)

    pos = nx.circular_layout(g)
    nx.draw(g, pos, with_labels=True, arrowsize=20, node_size=700, node_color='skyblue')

    return m

pi = [2, 1, 4, 5, 3]

print("connections:")
connections = plot_all_connections(pi)
print(connections)
print("usage matrix from recursive function:")
print(plot_all_MISS(pi))
print("usage matrix from connections and networkx:")
print(MISS_from_connections(connections, len(pi)))

___
### Taking a Closer Look at the Recursive Function

With the previous permutation of $1,\dots,5$:

$$
\pi = 2\ 1\ 4\ 5\ 3
$$

in the function (without plotting)

```python
def find_sources(s):
    """finds sources of a sequence (all sources of MISS)

    Args:
        s (list): main sequence

    Returns:
        list: indices of source nodes
    """
    return [i for i, element in enumerate(s) if all(element < x for x in s[:i])]

def recursive_function(s):
    """calls recursive function to find all MISS

    Args:
        s (list): main sequence

    Returns:
        2D numpy array: usage matrix MISS
    """
    m = np.empty((0, len(s)))
    def recurse(s, x, row):
        row[x] = 1
        greater = []
        for i in range(x, len(s)):
            if s[i] > s[x] and all(s[j] > s[i] for j in greater):
                greater.append(i)
        if len(greater) == 0:
            nonlocal m
            m = np.concatenate((m, [row]), axis=0)
            return
        for element in greater:
            recurse(s, element, row.copy())

    for source in find_sources(s):
        recurse(s, source, np.zeros(len(s))) 

    return m

pi = [2, 1, 4, 5, 3]
recursive_function(pi)
```
`find_sources` returns a list of incides of the "sources" of the sequence. For MISS, these are the indices `i` of `s` such that every element of index less than `i` has value greater than `s[i]`.

The call `recursive_function(pi)` calls the outside wrapper for `recurse` with `s` as the given sequence $\pi$. \
This initializes the return usage matrix m as an empty numpy matrix. Then, for every source index in the source list, `recurse` is called. `recurse` can be thought of as a form of depth first search. With the previous example, the sources are at indices 0 and 1. These are the base "branches."

Starting with 0, `recurse` is called with `s`, 0, and [0 0 0 0 0]. Recurse sets `s[0]` to 1 so the `row` is now [1 0 0 0 0]. Now initialize `greater` as an empty list. for every element from the current index to the end of the sequence, we check them in order with the statement `if s[i] > s[x] and all(s[j] > s[i] for j in greater)`. In essence, this statement checks if all elements already added to the `greater` list are greater than the current element. If so, the index of this element can be added to `greater`. This is because if there is an element in `greater` that is less than the current element, the connection from `s[x]` to the current element will *not* be part of a MISS becuase it can be easily be extended by going through an element in `greater`. Continuing from the example, `greater` starts as [], then [2], then [2, 4]. Then we check if `greater` is empty. In this case, it is not. Finally, we call `recurse` on every element in `greater` with the given row [1 0 0 0 0].

Now going through futher calls in the recursion, `recurse(s, 0, [0 0 0 0 0])` calls `recurse(s, 2, [1 0 0 0 0])` calls `recurse(s, 3, [1 0 1 0 0])`. Here, the row is set as [1 0 1 1 0] and greater is []. Thus, this row is then concatenated on the matrix. `m` is now [1 0 1 1 0] and we return. This returns again as we are out of elements in `greater`. Back to the first recursive call, we still have index 4 to go through. This results in row [1 0 0 0 1] being appended to m. We then go through the next source, index 1, recursing and concatenating as described before, until the entire matrix `m` is generated and we return `m` from the recursive wrapper function.


___
Now consider the permutation of $1,\dots,20$:

$$
\pi = 7\ 3\ 1\ 17\ 10\ 18\ 9\ 20\ 6\ 12\ 16\ 13\ 2\ 8\ 19\ 4\ 15\ 11\ 14\ 5
$$

In [ ]:
pi =  [7, 3, 1, 17, 10, 18, 9, 20, 6, 12, 16, 13, 2, 8, 19, 4, 15, 11, 14, 5]

x = range(1,21)

# initialize figure
fig, fig = plt.subplots(figsize=(6, 6))
    
plt.scatter(x,pi)

# Set the ticks to be at the edges of the bins.
fig.set_xticks(range(0,21))
fig.set_yticks(range(0,21))
plt.grid(True)

Here's an example of a **MISS** (also maximum):
$$
1\ 6\ 8\ 11\ 14
$$
corresponding to the indicies $3<9<14<18<19$.

In [ ]:
pi =  [7, 3, 1, 17, 10, 18, 9, 20, 6, 12, 16, 13, 2, 8, 19, 4, 15, 11, 14, 5]
x = range(1,21)

ga = [1,6,8,11,14]
I = [3,9,14,18,19]

# initialize figure
fig, fig = plt.subplots(figsize=(6, 6))
    
plt.scatter(x,pi)
plt.plot(I,ga)

# Set the ticks to be at the edges of the bins.
fig.set_xticks(range(0,21))
fig.set_yticks(range(0,21))
plt.grid(True)

Now to find all MISS.

In [ ]:
def plot_all_connections(s):
    """returns all valid connections in MISS

    Args:
        s (list): sequence to generate connections from

    Returns:
        list of tuples: (x1, x2) where (x1, s[x1]) (x2, s[x2]) is a valid connection of up and right paths that are part of a MISS
    """
    connections = []
    for i in range(len(s)):
        for j in range(i, len(s)):
            # using the above rule
            if ((j - i) * (s[j] - s[i])) > 0:
                connections.append((i, j))
    # gets rid of duplicates
    connections = list(set(connections))

    plt.figure(figsize=(6, 6))
    plt.scatter(range(1, len(s) + 1), s)

    valid_connections = set()

    for connection in connections:
        if not in_box(connection, s):
            valid_connections.add((connection[0] + 1, connection[1] + 1))
            x1, x2 = connection[0], connection[1]
            plt.plot([x1 + 1, x2 + 1], [s[x1], s[x2]], color='tan')

    plt.xticks(range(0, len(s) + 1))
    plt.yticks(range(0, len(s) + 1))
    plt.grid(True)

    connections = sorted(list(valid_connections))

    return connections

def in_box(t, s):
    """returns whether there is another point in the box extending horizontal and vertical lines from both points

    Args:
        t (tuple): connection to be checked
        s (list): main seequence

    Returns:
        bool: whether there exists a point in the box
    """
    i, j = t[0], t[1]
    if any(s[x] < s[j] and s[x] > s[i] for x in range(i + 1, j)):
        return True
    return False

def find_sources(s):
    """finds sources of a sequence (all sources of MISS)

    Args:
        s (list): main sequence

    Returns:
        list: indices of source nodes
    """
    return [i for i, element in enumerate(s) if all(element < x for x in s[:i])]

def plot_all_MISS(s):
    """calls recursive function to find all MISS

    Args:
        s (list): main sequence

    Returns:
        2D numpy array: usage matrix MISS
    """
    m = np.empty((0, len(s)))
    def recurse(s, x, row):
        row[x] = 1
        greater = []
        for i in range(x, len(s)):
            if s[i] > s[x] and all(s[j] > s[i] for j in greater):
                greater.append(i)
        if len(greater) == 0:
            nonlocal m
            m = np.concatenate((m, [row]), axis=0)
            return
        for element in greater:
            recurse(s, element, row.copy())

    for source in find_sources(s):
        recurse(s, source, np.zeros(len(s))) 

    plt.figure(figsize=(6, 6))

    plt.scatter(range(1, len(s) + 1), s)
    
    for row in m:
        indices = np.nonzero(row)
        plt.plot(indices[0] + 1, [s[i] for i in indices[0]], color='tan')

    plt.xticks(range(0, len(s) + 1))
    plt.yticks(range(0, len(s) + 1))
    plt.grid(True)

    return m

def MISS_from_connections(c, s_len):
    """uses networkx to find the usage matrix of MISS from a list of possible valid connections

    Args:
        c (list of tuples): connections

    Returns:
        2D numpy array: usage matrix
    """
    plt.figure(figsize=(6, 6))
    g = nx.DiGraph()
    g.add_edges_from(c)
    
    m = np.empty((0, s_len))
        
    sink_nodes = [node for node, out_degree in g.out_degree() if out_degree == 0]
    source_nodes = [node for node, in_degree in g.in_degree() if in_degree == 0]
    for (source, sink) in [(source, sink) for sink in sink_nodes for source in source_nodes]:
        for path in nx.all_simple_paths(g, source=source, target=sink):
            row = np.zeros(s_len)
            for i in path:
                row[i - 1] = 1
            m = np.concatenate((m, [row]), axis=0)

    pos = nx.circular_layout(g)
    nx.draw(g, pos, with_labels=True, arrowsize=20, node_size=700, node_color='skyblue')

    return m

pi =  [7, 3, 1, 17, 10, 18, 9, 20, 6, 12, 16, 13, 2, 8, 19, 4, 15, 11, 14, 5]

print("connections:")
connections = plot_all_connections(pi)
print(connections)
print("usage matrix from recursive function:")
print(plot_all_MISS(pi))
print("usage matrix from connections and networkx:")
print(MISS_from_connections(connections, len(pi)))

___
### Verifying and Timing Algorithms

refactoring functions to remove plotting \
ordering rows and checking for equality \
using `%timeit` to time

In [ ]:
def all_connections(s):
    """returns all valid connections in MISS

    Args:
        s (list): sequence to generate connections from

    Returns:
        list of tuples: (x1, x2) where (x1, s[x1]) (x2, s[x2]) is a valid connection of up and right paths that are part of a MISS
    """
    connections = []
    for i in range(len(s)):
        for j in range(i, len(s)):
            # using the above rule
            if ((j - i) * (s[j] - s[i])) > 0:
                connections.append((i, j))
    # gets rid of duplicates
    connections = list(set(connections))

    valid_connections = set()

    for connection in connections:
        if not in_box(connection, s):
            valid_connections.add((connection[0] + 1, connection[1] + 1))

    connections = sorted(list(valid_connections))

    return connections

def in_box(t, s):
    """returns whether there is another point in the box extending horizontal and vertical lines from both points

    Args:
        t (tuple): connection to be checked
        s (list): main seequence

    Returns:
        bool: whether there exists a point in the box
    """
    i, j = t[0], t[1]
    if any(s[x] < s[j] and s[x] > s[i] for x in range(i + 1, j)):
        return True
    return False

def find_sources(s):
    """finds sources of a sequence (all sources of MISS)

    Args:
        s (list): main sequence

    Returns:
        list: indices of source nodes
    """
    return [i for i, element in enumerate(s) if all(element < x for x in s[:i])]

def recursive_method(s):
    """calls recursive function to find all MISS

    Args:
        s (list): main sequence

    Returns:
        2D numpy array: usage matrix MISS
    """
    m = np.empty((0, len(s)))
    def recurse(s, x, row):
        row[x] = 1
        greater = []
        for i in range(x, len(s)):
            if s[i] > s[x] and all(s[j] > s[i] for j in greater):
                greater.append(i)
        if len(greater) == 0:
            nonlocal m
            m = np.concatenate((m, [row]), axis=0)
            return
        for element in greater:
            recurse(s, element, row.copy())

    for source in find_sources(s):
        recurse(s, source, np.zeros(len(s))) 
    return m

def MISS_from_connections(c, s_len):
    """uses networkx to find the usage matrix of MISS from a list of possible valid connections

    Args:
        c (list of tuples): connections

    Returns:
        2D numpy array: usage matrix
    """
    g = nx.DiGraph()
    g.add_edges_from(c)
    
    m = np.empty((0, s_len))
        
    sink_nodes = [node for node, out_degree in g.out_degree() if out_degree == 0]
    source_nodes = [node for node, in_degree in g.in_degree() if in_degree == 0]
    for (source, sink) in [(source, sink) for sink in sink_nodes for source in source_nodes]:
        for path in nx.all_simple_paths(g, source=source, target=sink):
            row = np.zeros(s_len)
            for i in path:
                row[i - 1] = 1
            m = np.concatenate((m, [row]), axis=0)

    return m

def networkx_method(pi):
    return MISS_from_connections(all_connections(pi), len(pi))

def improved_recursive(s):
    m_rows = []

    def recurse(s, x, row):
        row[x] = 1
        greater = []
        for i in range(x, len(s)):
            if s[i] > s[x] and all(s[j] > s[i] for j in greater):
                greater.append(i)
        if not greater:
            m_rows.append(row.copy())
            return
        for element in greater:
            recurse(s, element, row)
            row[element] = 0
            
    for i, element in enumerate(s):
        if all(element < x for x in s[:i]):
            recurse(s, i, np.zeros(len(s)))
        
    return np.array(m_rows)

pi = [7, 3, 1, 17, 10, 18, 9, 20, 6, 12, 16, 13, 2, 8, 19, 4, 15, 11, 14, 5]
# pi = [2, 1, 4, 5, 3]

print("usage matrix using recursive function:")
r = recursive_method(pi)
r = r[np.lexsort(r.T)]
print(r)
print("usage matrix using networkx method:")
n = networkx_method(pi)
n = n[np.lexsort(n.T)]
print(n)
print("usage matrix using improved recursive function:")
i = improved_recursive(pi)
i = i[np.lexsort(i.T)]
print(i)

print(f"Outputs of all methods are equal: {np.array_equal(n, r) and np.array_equal(n, i)}")

In [ ]:
pi = [7, 3, 1, 17, 10, 18, 9, 20, 6, 12, 16, 13, 2, 8, 19, 4, 15, 11, 14, 5]
print("recursive method: ", end="")
%timeit -n 1000 recursive_method(pi) 
print("networkx method: ", end="")
%timeit -n 1000 networkx_method(pi)
print("improved recursive method: ", end="")
%timeit -n 1000 improved_recursive(pi)

___
### Find all MDSS - Examples

(maximal decreasing subsequences)

Of course, the resulting graph will be a subgraph of the graph obtained by saying that two points $(i_1,\pi(i_1))$ and $(i_2,\pi(i_2))$ are connected iff $(i_2-i_1)(\pi(i_2)-\pi(i_1))<0$

Consider the same permutation of $1,\dots,5$:

$$
\pi = 2\ 1\ 4\ 5\ 3
$$

In [ ]:
def plot_all_connections(s):
    """returns all valid connections in MDSS

    Args:
        s (list): sequence to generate connections from

    Returns:
        list of tuples: (x1, x2) where (x1, s[x1]) (x2, s[x2]) is a valid connection of down and left paths that are part of a MISS
    """
    connections = []
    for i in range(len(s)):
        for j in range(i, len(s)):
            # using the above rule
            if ((j - i) * (s[j] - s[i])) < 0:
                connections.append((i, j))
    # gets rid of duplicates
    connections = list(set(connections))

    plt.figure(figsize=(4, 4))
    plt.scatter(range(1, len(s) + 1), s)

    valid_connections = set()

    for connection in connections:
        if not in_box(connection, s):
            valid_connections.add((connection[0] + 1, connection[1] + 1))
            x1, x2 = connection[0], connection[1]
            plt.plot([x1 + 1, x2 + 1], [s[x1], s[x2]], color='tan')

    plt.xticks(range(0, len(s) + 1))
    plt.yticks(range(0, len(s) + 1))
    plt.grid(True)

    connections = sorted(list(valid_connections))

    return connections

def in_box(t, s):
    """returns whether there is another point in the box extending horizontal and vertical lines from both points

    Args:
        t (tuple): connection to be checked
        s (list): main seequence

    Returns:
        bool: whether there exists a point in the box
    """
    i, j = t[0], t[1]
    if any(s[x] > s[j] and s[x] < s[i] for x in range(i + 1, j)):
        return True
    return False

def MISS_from_connections(c, s_len):
    """uses networkx to find the usage matrix of MISS from a list of possible valid connections

    Args:
        c (list of tuples): connections

    Returns:
        2D numpy array: usage matrix
    """
    plt.figure(figsize=(4, 4))
    g = nx.DiGraph()
    g.add_edges_from(c)
    
    m = np.empty((0, s_len))
        
    sink_nodes = [node for node, out_degree in g.out_degree() if out_degree == 0]
    source_nodes = [node for node, in_degree in g.in_degree() if in_degree == 0]
    for (source, sink) in [(source, sink) for sink in sink_nodes for source in source_nodes]:
        for path in nx.all_simple_paths(g, source=source, target=sink):
            row = np.zeros(s_len)
            for i in path:
                row[i - 1] = 1
            m = np.concatenate((m, [row]), axis=0)

    pos = nx.circular_layout(g)
    nx.draw(g, pos, with_labels=True, arrowsize=20, node_size=700, node_color='skyblue')

    return m

pi = [2, 1, 4, 5, 3]

print("connections:")
connections = plot_all_connections(pi)
print(connections)
print("usage matrix from connections and networkx:")
print(MISS_from_connections(connections, len(pi)))

Again, the same permutation of $1,\dots,20$:

$$
\pi = 7\ 3\ 1\ 17\ 10\ 18\ 9\ 20\ 6\ 12\ 16\ 13\ 2\ 8\ 19\ 4\ 15\ 11\ 14\ 5
$$

In [ ]:
def plot_all_connections(s):
    """returns all valid connections in MDSS

    Args:
        s (list): sequence to generate connections from

    Returns:
        list of tuples: (x1, x2) where (x1, s[x1]) (x2, s[x2]) is a valid connection of down and left paths that are part of a MISS
    """
    connections = []
    for i in range(len(s)):
        for j in range(i, len(s)):
            # using the above rule
            if ((j - i) * (s[j] - s[i])) < 0:
                connections.append((i, j))
    # gets rid of duplicates
    connections = list(set(connections))

    plt.figure(figsize=(6, 6))
    plt.scatter(range(1, len(s) + 1), s)

    valid_connections = set()

    for connection in connections:
        if not in_box(connection, s):
            valid_connections.add((connection[0] + 1, connection[1] + 1))
            x1, x2 = connection[0], connection[1]
            plt.plot([x1 + 1, x2 + 1], [s[x1], s[x2]], color='tan')

    plt.xticks(range(0, len(s) + 1))
    plt.yticks(range(0, len(s) + 1))
    plt.grid(True)

    connections = sorted(list(valid_connections))

    return connections

def in_box(t, s):
    """returns whether there is another point in the box extending horizontal and vertical lines from both points

    Args:
        t (tuple): connection to be checked
        s (list): main seequence

    Returns:
        bool: whether there exists a point in the box
    """
    i, j = t[0], t[1]
    if any(s[x] > s[j] and s[x] < s[i] for x in range(i + 1, j)):
        return True
    return False

def MISS_from_connections(c, s_len):
    """uses networkx to find the usage matrix of MISS from a list of possible valid connections

    Args:
        c (list of tuples): connections

    Returns:
        2D numpy array: usage matrix
    """
    plt.figure(figsize=(6, 6))
    g = nx.DiGraph()
    g.add_edges_from(c)
    
    m = np.empty((0, s_len))
        
    sink_nodes = [node for node, out_degree in g.out_degree() if out_degree == 0]
    source_nodes = [node for node, in_degree in g.in_degree() if in_degree == 0]
    for (source, sink) in [(source, sink) for sink in sink_nodes for source in source_nodes]:
        for path in nx.all_simple_paths(g, source=source, target=sink):
            row = np.zeros(s_len)
            for i in path:
                row[i - 1] = 1
            m = np.concatenate((m, [row]), axis=0)

    pos = nx.circular_layout(g)
    nx.draw(g, pos, with_labels=True, arrowsize=20, node_size=700, node_color='skyblue')

    return m

pi =  [7, 3, 1, 17, 10, 18, 9, 20, 6, 12, 16, 13, 2, 8, 19, 4, 15, 11, 14, 5]

print("connections:")
connections = plot_all_connections(pi)
print(connections)
print("usage matrix from connections and networkx:")
print(MISS_from_connections(connections, len(pi)))

___
Giving each node a weight and finding the minimal weight MISS with
$$
\pi = 7\ 3\ 1\ 17\ 10\ 18\ 9\ 20\ 6\ 12\ 16\ 13\ 2\ 8\ 19\ 4\ 15\ 11\ 14\ 5
$$
and node weights
$$
w = .1\ .3\ .4\ .2\ .5\ .7\ .2\ .3\ .9\ .6\ .3\ .6\ .1\ .2\ .4\ .3\ .4\ .8\ .9\ .2\
$$

In [ ]:
def find_sources(s):
    """finds sources of a sequence (all sources of MISS)

    Args:
        s (list): main sequence

    Returns:
        list: indices of source nodes
    """
    return [i for i, element in enumerate(s) if all(element < x for x in s[:i])]

def lightest_row(s, w):
    """calls recursive function to find all MISS

    Args:
        s (list): main sequence

    Returns:
        2D numpy array: usage matrix MISS
    """
    m = np.empty((0, len(s)))
    lightest = np.empty(len(s))
    lightest_value = math.inf
    def recurse(s, x, row):
        row[x] = 1
        greater = []
        for i in range(x, len(s)):
            if s[i] > s[x] and all(s[j] > s[i] for j in greater):
                greater.append(i)
        if len(greater) == 0:
            nonlocal m, lightest, lightest_value
            m = np.concatenate((m, [row]), axis=0)
            value = np.dot(row, w)
            if value < lightest_value:
                lightest_value = value
                lightest = row
            return
        for element in greater:
            recurse(s, element, row.copy())

    for source in find_sources(s):
        recurse(s, source, np.zeros(len(s))) 

    plt.figure(figsize=(6, 6))

    plt.scatter(range(1, len(s) + 1), s)
    
    for row in m:
        indices = np.nonzero(row)
        plt.plot(indices[0] + 1, [s[i] for i in indices[0]], color='tan')

    plt.xticks(range(0, len(s) + 1))
    plt.yticks(range(0, len(s) + 1))
    plt.grid(True)

    indices = np.nonzero(lightest)
    plt.plot(indices[0] + 1, [s[i] for i in indices[0]], color='darkblue')

    return m, lightest, lightest_value

pi =  [ 7,  3,  1, 17, 10, 18,  9, 20,  6, 12, 16, 13,  2,  8, 19,  4, 15, 11, 14,  5]
rho = [.1, .3, .4, .2, .5, .7, .2, .3, .9, .6, .3, .6, .1, .2, .4, .3, .4, .8, .9, .2]

m, lightest, lv = lightest_row(pi, rho)
print("lightest row:")
print(f"{lightest}, value {round(lv, 15)}")

### Creating Balanced Binary Search Tree

In [ ]:
#helper image
pi =  [7, 3, 1, 17, 10, 18, 9, 20, 6, 12, 16, 13, 2, 8, 19, 4, 15, 11, 14, 5]

x = range(1,21)

# initialize figure
fig, fig = plt.subplots(figsize=(5, 5))
    
plt.scatter(x,pi)

# Set the ticks to be at the edges of the bins.
fig.set_xticks(range(0,21))
fig.set_yticks(range(0,21))
plt.grid(True)

<img src="tree.jpeg" alt="AVL Tree for pi" style="width: 700px;"/>

In [ ]:
#small helper image
pi =  [2, 1, 4, 5, 3]
x = range(1,6)

# initialize figure
fig, fig = plt.subplots(figsize=(2, 2))
    
plt.scatter(x,pi)

# Set the ticks to be at the edges of the bins.
fig.set_xticks(range(0,6))
fig.set_yticks(range(0,6))
plt.grid(True)

<img src="small_tree.jpeg" alt="AVL Tree for pi" style="width: 200px;"/>

In [ ]:
class Node(): 
    def __init__(self, val): 
        self.val = val 
        self.left = None
        self.right = None
        self.height = 1
  
class AVL_Tree(): 
    def insert(self, node, val): 
        if not node: 
            return Node(val) 
        elif val < node.val: 
            node.left = self.insert(node.left, val) 
        else: 
            node.right = self.insert(node.right, val) 
  
        node.height = 1 + max(self.height(node.left), 
                           self.height(node.right)) 
  
        balanced = self.balance(node) 

        # single right rotate
        if balanced > 1 and val < node.left.val: 
            return self.rightRotate(node) 

        # single left rotate
        if balanced < -1 and val > node.right.val: 
            return self.leftRotate(node) 
        
        # double right rotate
        if balanced > 1 and val > node.left.val: 
            node.left = self.leftRotate(node.left) 
            return self.rightRotate(node) 

        # double left rotate
        if balanced < -1 and val < node.right.val: 
            node.right = self.rightRotate(node.right) 
            return self.leftRotate(node) 

        return node 
  
    def leftRotate(self, node): 
        right_child = node.right 
        t = right_child.left 
  
        right_child.left = node 
        node.right = t 
  
        node.height = 1 + max(self.height(node.left), self.height(node.right)) 
        right_child.height = 1 + max(self.height(right_child.left), self.height(right_child.right)) 
  
        return right_child 
  
    def rightRotate(self, node): 
        left_child = node.left 
        t = left_child.right 
  
        left_child.right = node 
        node.left = t 
  
        node.height = 1 + max(self.height(node.left), self.height(node.right)) 
        left_child.height = 1 + max(self.height(left_child.left), self.height(left_child.right)) 
        
        return left_child 
  
    def height(self, node): 
        if not node: 
            return 0
        return node.height 
  
    def balance(self, node): 
        if not node: 
            return 0
        return self.height(node.left) - self.height(node.right) 
    
    def preOrder(self, root):

        def preO(root): 
            if not root: 
                return
            print(root.val, end=" ") 
            preO(root.left) 
            preO(root.right) 

        preO(root)
        print()
        

pi = [7, 3, 1, 17, 10, 18, 9, 20, 6, 12, 16, 13, 2, 8, 19, 4, 15, 11, 14, 5]

pi_short = [2, 1, 4, 5, 3]

pi_increasing = [x for x in range(1, 6)] # 1, 2, 3, 4, 5

t = AVL_Tree()
root = None

for m in pi_increasing:
    root = t.insert(root, m)

t.preOrder(root)

root = None

for m in pi:
    root = t.insert(root, m)

t.preOrder(root)



#### Small Simulation

Simulating AVL tree creation on `pi_increasing`, 1 starts as the root. Then 2 is inserted as the right child of 1. Then 3 is inserted as the right child of 2. But now with 1 being the root, the left side has height 0 and the right side has height 2. Therefore, balanced = -2 and a left rotate is performed. Now the root is 2 with 1 as its left child and 3 as its right child. Then 4 is inserted as the right child of 3. Then 5 is inserted as the right child of 4 and a left rotate is performed on node 3.

### Augmented Binary Search Tree

note: needs split and concatenate for avl trees when splitting $Q^{*}$  $Q_{\alpha}$, also balance from the last implementation, also pointer to parent, also max?
and also SPLIT and CONCATENATE. also figure out when insertion happens. and make sure is actually balanced.

Read section 8 and 9 again

height should probably be changed to be part of a node and not a rerunning function every time

In [ ]:
class Node(): 
    def __init__(self, val, internal=False): 
        self.val = val
        self.f = None
        self.left = None
        self.right = None
        if not internal:
            self.q = Node(val, True)
  
class Augmented_Tree(): 
    def insert(self, node, val, parent=None): 
        if not node: 
            new_node = Node(val)
            new_node.f = parent
            return new_node
        elif val < node.val: 
            node.left = self.insert(node.left, val, node) 
        else: 
            node.right = self.insert(node.right, val, node) 

        node = self.balance(node)

        return node 
    
    def avlsplit(self, node, k):
        if not node: 
            return None, False, None
        l, m, r = node.left, node.val, node.right
        if k == m:
            new_l = Node(m)
            new_l.left = l
            if l:
                l.f = new_l
            r.f = None
            return self.balance(new_l), True, r
        if k < m:
            lprime, b, rprime = self.avlsplit(l, k)
            return lprime, b, self.avljoin(rprime, m, r)
        if k > m:
            lprime, b, rprime = self.avlsplit(r, k)
            return self.avljoin(l, m, lprime), b, rprime

    def avlrightjoin(self, left, k, right):
        l, kprime, c = left.left, left.val, left.right
        if self.height(c) <= self.height(right) + 1:
            tprime = Node(k)
            tprime.left = c
            if c:
                c.f = tprime
            tprime.right = right
            if right:
                right.f = tprime
            if self.height(tprime) <= self.height(l) + 1:
                return_node = Node(kprime)
                return_node.left = l
                if l:
                    l.f = return_node
                return_node.right = tprime
                if tprime:
                    tprime.f = return_node
                return return_node
            else: 
                return_node = Node(kprime)
                return_node.left = l
                if l:
                    l.f = return_node
                return_node.right = self.rightRotate(tprime)
                return self.leftRotate(return_node)
        else:
            tprime = self.avlrightjoin(c, k, right)
            tprimeprime = Node(kprime)
            tprimeprime.left = l
            if l:
                l.f = tprimeprime
            tprimeprime.right = tprime
            if tprime:
                tprime.f = tprimeprime
            if self.height(tprime) <= self.height(l) + 1:
                return tprimeprime
            else:
                return self.leftRotate(tprimeprime)
            
    def avlleftjoin(self, left, k, right):
        c, kprime, r = right.left, right.val, right.right
        if self.height(c) <= self.height(left) + 1:
            tprime = Node(k)
            tprime.left = left
            if left:
                left.f = tprime
            tprime.right = c
            if c:
                c.f = tprime
            if self.height(tprime) <= self.height(r) + 1:
                return_node = Node(kprime)
                return_node.left = tprime
                if tprime:
                    tprime.f = return_node
                return_node.right = r
                if r:
                    r.f = return_node
                return return_node
            else:
                return_node = Node(kprime)
                return_node.left = self.leftRotate(tprime)
                return_node.right = r
                if r:
                    r.f = return_node
                return self.rightRotate(return_node)
        else:
            tprime = self.avlleftjoin(left, k, c)
            tprimeprime = Node(kprime)
            tprimeprime.left = tprime
            if tprime:
                tprime.f = tprimeprime
            tprimeprime.right = r
            if r:
                r.f = tprimeprime
            if self.height(tprime) <= self.height(r) + 1:
                return tprimeprime
            else:
                return self.rightRotate(tprimeprime)

    # requires that k > all elements in left and k < all elements in right
    def avljoin(self, left, k, right):
        if self.height(left) > self.height(right) + 1:
            return self.avlrightjoin(left, k, right)
        if self.height(right) > self.height(left) + 1:
            return self.avlleftjoin(left, k, right)
        return_node = Node(k)
        return_node.left = left
        if left:
            left.f = return_node
        return_node.right = right
        if right:
            right.f = return_node
        return return_node
    
    def avlunion(self, node1, node2):
        if not node1:
            return node2
        if not node2:
            return node1
        tless, b, tgreater = self.avlsplit(node2, node1.val)
        return self.avljoin(self.avlunion(node1.left, tless), node1.val, self.avlunion(node1.right, tgreater))

    def split(self):
        pass

    def concatenate(self):
        pass

    def down(self, a, b):
        if a.val == b.val:
            return a
        else:
            q1, q2 = self.avlsplit(a.q, a.b)
            a.left.q = self.avlunion(a.left.q, q1)
            a.right.q = self.avlunion(a.right.q, q2)
            if b.val < a.val:
                self.down(a.left, b)
            else:
                self.down(a.right, b)

    def up(self, a):
        pass

    def balance(self, node):
        balanced = self.height(node.left) - self.height(node.right) 

        # single right rotate
        if balanced > 1 and val < node.left.val: 
            return self.rightRotate(node) 

        # single left rotate
        if balanced < -1 and val > node.right.val: 
            return self.leftRotate(node) 
        
        # double right rotate
        if balanced > 1 and val > node.left.val: 
            node.left = self.leftRotate(node.left) 
            return self.rightRotate(node) 

        # double left rotate
        if balanced < -1 and val < node.right.val: 
            node.right = self.rightRotate(node.right) 
            return self.leftRotate(node) 
        
        return node

    def height(self, node):
        if not node:
            return 0
        return 1 + max(self.height(node.left), self.height(node.right))
    
    def leftRotate(self, node): 
        tmp = node.right
        node.right = tmp.left
        if tmp.left:
            tmp.left.f = node
        tmp.left = node
        tmp.f = node.f
        node.f = tmp
        if tmp.f:
            if node.val < tmp.f.val:
                tmp.f.left = tmp
            else:
                tmp.f.right = tmp
        return tmp
  
    def rightRotate(self, node): 
        tmp = node.left
        node.left = tmp.right
        if tmp.right:
            tmp.right.f = node
        tmp.right = node
        tmp.f = node.f
        node.f = tmp
        if tmp.f:
            if node.val < tmp.f.val:
                tmp.f.left = tmp
            else:
                tmp.f.right = tmp
        return tmp

    def preOrder(self, root):
        def preO(node): 
            if not node: 
                return
            print(f'Node: {node.val}, Parent: {node.f.val if node.f else None}') 
            preO(node.left) 
            preO(node.right)
        preO(root)
        print()
  

# pi = [7, 3, 1, 17, 10, 18, 9, 20, 6, 12, 16, 13, 2, 8, 19, 4, 15, 11, 14, 5]

# pi_short = [2, 1, 4, 5, 3]

pi_increasing1 = [x for x in range(1, 6)] # 1, 2, 3, 4, 5
pi_increasing2 = [x for x in range(6, 8)] # 6, 7

t = Augmented_Tree()
root1 = None
for val in pi_increasing1:
    root1 = t.insert(root1, val)
t.preOrder(root1)

root2 = None
for val in pi_increasing2:
    root2 = t.insert(root2, val)
t.preOrder(root2)

root3 = t.avlunion(root1, root2)
t.preOrder(root3)

root4, n, root5 = t.avlsplit(root3, 4)
t.preOrder(root4)
t.preOrder(root5)

root6 = t.avlunion(root4, root5)
t.preOrder(root6)



### Implementing Basic Algorithm


In [ ]:
def find_sources(s):
    return [i for i, element in enumerate(s) if all(element < x for x in s[:i])]

def shortest_path(s, w):
    m = np.empty((0, len(s)))
    lightest = np.empty(len(s))
    lightest_value = math.inf
    def recurse(s, x, row):
        row[x] = 1
        greater = []
        for i in range(x, len(s)):
            if s[i] > s[x] and all(s[j] > s[i] for j in greater):
                greater.append(i)
        if len(greater) == 0:
            nonlocal m, lightest, lightest_value
            m = np.concatenate((m, [row]), axis=0)
            value = np.dot(row, w)
            if value < lightest_value:
                lightest_value = value
                lightest = row
            return
        for element in greater:
            recurse(s, element, row.copy())

    for source in find_sources(s):
        recurse(s, source, np.zeros(len(s))) 

    return m, lightest, lightest_value

def add_constraint(N, c):
    if N is None:
        return c
    return np.vstack((N, c))

def modulus(N, p):
    n = N.shape[-1]

    rho = cvx.Variable(n)

    cons = [rho >= 0, N@rho >= 1]

    obj = cvx.Minimize(cvx.pnorm(1**(1/p)*rho, p)**p)

    prob = cvx.Problem(obj, cons)
    prob.solve(solver=cvx.ECOS)

    return obj.value, np.array(rho.value).flatten()

pi = [7, 3, 1, 17, 10, 18, 9, 20, 6, 12, 16, 13, 2, 8, 19, 4, 15, 11, 14, 5]

pi_short = [2, 1, 4, 5, 3]

pi_increasing = [x for x in range(1, 6)]

pi_decreasing = [x for x in range(5, 0, -1)]

pi_test = [2, 1, 3, 4, 5]

pi_test_2 = [2, 1, 3, 5, 4]

def ssmodulus(pi, p):
    rho = np.zeros(len(pi))
    mod = 0
    tol = 1e-5
    N = None

    while True:
        m, gP, lv = shortest_path(pi, rho)

        if lv > 1 - tol:
            plt.figure(figsize=(6, 6))
            plt.scatter(range(1, len(pi) + 1), pi)
            plt.title(f'Modulus: {round(mod, 3)}')
            for row in m:
                indices = np.nonzero(row)
                plt.plot(indices[0] + 1, [pi[i] for i in indices[0]], color='tan')
            for i, j in enumerate(pi):
                plt.annotate(round(rho[i], 3), (i + 1, j), xycoords='data', xytext=(4, 4), textcoords='offset points')
            plt.xticks(range(0, len(pi) + 2))
            plt.yticks(range(0, len(pi) + 2))
            plt.grid(True)
            return mod, rho
        
        N = add_constraint(N, gP)

        mod, rho = modulus(N, p)

# one modulus
# print(ssmodulus(pi_increasing, 1))
# print(ssmodulus(pi_decreasing, 1))
# print(ssmodulus(pi_short, 1))
# print(ssmodulus(pi_test, 1))
# print(ssmodulus(pi_test_2, 1))
# print(ssmodulus(pi, 1))

# two modulus
print(ssmodulus(pi_increasing, 2))
print(ssmodulus(pi_decreasing, 2))
print(ssmodulus(pi_short, 2))
print(ssmodulus(pi_test, 2))
print(ssmodulus(pi_test_2, 2))
print(ssmodulus(pi, 2))

### Plus one algorithm

In [ ]:
pi = [7, 3, 1, 17, 10, 18, 9, 20, 6, 12, 16, 13, 2, 8, 19, 4, 15, 11, 14, 5]

pi_short = [2, 1, 4, 5, 3]

pi_increasing = [x for x in range(1, 6)]

pi_test = [2, 1, 3, 4, 5]

pi_test_2 = [2, 1, 3, 5, 4]

def ssmodulus_plus_one(pi, num_iter):
    mod = 0
    n = np.zeros(len(pi))

    for it in range(num_iter):
        m, gP, lv = shortest_path(pi, n)
        for x in np.nonzero(gP):
            # n[x] += 1/len(x)
            n[x] += 1

    # n = n.T * n / num_iter**2
    n = n / num_iter
    mod = 1. / sum(i ** 2 for i in n)

    plt.figure(figsize=(6, 6))
    plt.scatter(range(1, len(pi) + 1), pi)
    plt.title(f'Modulus: {round(mod, 3)}')
    for row in m:
        indices = np.nonzero(row)
        plt.plot(indices[0] + 1, [pi[i] for i in indices[0]], color='tan')
    for i, j in enumerate(pi):
        plt.annotate(round(n[i], 3), (i + 1, j), xycoords='data', xytext=(4, 4), textcoords='offset points')
    plt.xticks(range(0, len(pi) + 2))
    plt.yticks(range(0, len(pi) + 2))
    plt.grid(True)
    return mod, n

print(ssmodulus_plus_one(pi_increasing, 10000))
print(ssmodulus_plus_one(pi_short, 10000))
print(ssmodulus_plus_one(pi_test, 10000))
print(ssmodulus_plus_one(pi_test_2, 10000))
# print(ssmodulus_plus_one(pi, 10000))

___
### A comparison of algorithms

In [ ]:
def plot_both(pi, p=2, num_iter=10000):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    rho = np.zeros(len(pi))
    mod = 0
    tol = 1e-5
    N = None

    while True:
        m, gP, lv = shortest_path(pi, rho)

        if lv > 1 - tol:
            # plt.figure(figsize=(6, 6))
            ax1.scatter(range(1, len(pi) + 1), pi)
            ax1.set_title(f'Basic Algo Modulus: {round(mod, 3)}')
            for row in m:
                indices = np.nonzero(row)
                ax1.plot(indices[0] + 1, [pi[i] for i in indices[0]], color='tan')
            for i, j in enumerate(pi):
                ax1.annotate(round(rho[i], 3), (i + 1, j), xycoords='data', xytext=(4, 4), textcoords='offset points')
            ax1.set_xticks(range(0, len(pi) + 2))
            ax1.set_yticks(range(0, len(pi) + 2))
            ax1.grid(True)
            break
        
        N = add_constraint(N, gP)

        mod, rho = modulus(N, p)

    mod = 0
    n = np.zeros(len(pi))

    for it in range(num_iter):
        m, gP, lv = shortest_path(pi, n)
        for x in np.nonzero(gP):
            # n[x] += 1/len(x)
            n[x] += 1

    # n = n.T * n / num_iter**2
    n = n / num_iter
    mod = 1. / sum(i ** 2 for i in n)

    # plt.figure(figsize=(6, 6))
    ax2.scatter(range(1, len(pi) + 1), pi)
    ax2.set_title(f'Plus-1 Algo Modulus: {round(mod, 3)}')
    for row in m:
        indices = np.nonzero(row)
        ax2.plot(indices[0] + 1, [pi[i] for i in indices[0]], color='tan')
    for i, j in enumerate(pi):
        ax2.annotate(round(n[i], 3), (i + 1, j), xycoords='data', xytext=(4, 4), textcoords='offset points')
    ax2.set_xticks(range(0, len(pi) + 2))
    ax2.set_yticks(range(0, len(pi) + 2))
    ax2.grid(True)
    return



pi = [7, 3, 1, 17, 10, 18, 9, 20, 6, 12, 16, 13, 2, 8, 19, 4, 15, 11, 14, 5]

pi_short = [2, 1, 4, 5, 3]

pi_increasing = [x for x in range(1, 6)]

pi_test = [2, 1, 3, 4, 5]

pi_test_2 = [2, 1, 3, 5, 4]

plot_both(pi_short)
plot_both(pi_increasing)
plot_both(pi_test)
plot_both(pi_test_2)
plot_both(pi)


___
### Observation

Point labels of the Basic Algorithm represent corresponding entries in $\rho^*$, the optimal density.

Point lables of the Plus-1 Algorithm represent corresponding entries in $\eta^*$ where $\eta(v) = \mathbb{P}(v \in \text{MISS})$

They are related by $\eta^* = \frac{\rho^*}{\|\rho^*\|^2}$

Thus, even though Plus-1 only computes $\eta^*$, we can find the modulus with $\text{mod} = (\sum_{i \in \eta^*} \eta(i)^2)^{-1}$ seen in this line
```python 
mod = 1. / sum(i ** 2 for i in n)
```



### Notes for future

find modulus of DIS\
actually implement the Atallah Algorithm